### Restart and run all cells

### stock: dividend, price

In [1]:
import calendar
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

format_dict = {
               'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
               'price_x':'{:,.2f}','to_price':'{:,.2f}',
               'current':'{:,.2f}','div_amt':'{:,.2f}','amount':'{:,.2f}','yield':'{:,.2f}',
               'Q4':'{:,.2f}','Q3':'{:,.2f}','Q2':'{:,.2f}','Q1':'{:,.2f}','DIVIDEND':'{:,.2f}',
               'qty':'{:,}','shares':'{:,}','qty_z':'{:,}'
              }
pd.set_option("display.max_rows", None)

today = date.today()
today

datetime.date(2021, 12, 29)

In [2]:
sql = '''
SELECT *
FROM dividend
ORDER BY name'''
df_dividend = pd.read_sql(sql, const)
df_dividend.shape

(79, 13)

In [3]:
sql = '''
SELECT D.name, P.name
FROM dividend D
LEFT JOIN price P
ON D.name = P.name
AND P.date = "%s"
WHERE P.name IS NULL
'''
sql = sql % today
df_left = pd.read_sql(sql, const)
df_left

,name,name


### If above statement consists stock names, must delete from dividend as there are no price records

In [4]:
name = 'MCS'

In [5]:
sql = '''
SELECT *
FROM dividend
WHERE name = "%s"'''
sql = sql % name
df_dividend = pd.read_sql(sql, const)
df_dividend

,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,MCS,0.6,0.37,0.4,0.0,1.37,0.0,0.0,60000.0,2021-11-25,2021-12-09,None,0


In [6]:
sql = '''
SELECT *
FROM price
WHERE name = "%s"
ORDER BY date DESC'''
sql = sql % name
df_price = pd.read_sql(sql, const)
df_price.head()

,name,date,price,maxp,minp,qty,opnp
0,MCS,2021-12-29,14.8,14.8,14.6,2743214,14.8
1,MCS,2021-12-28,14.8,14.9,14.6,6949930,14.6
2,MCS,2021-12-27,14.5,14.7,14.4,3420083,14.7
3,MCS,2021-12-24,14.6,14.6,14.5,437599,14.6
4,MCS,2021-12-23,14.6,14.6,14.5,613259,14.6


In [7]:
sqlDel = '''
DELETE FROM dividend
WHERE name = "%s"'''
sqlDel = sqlDel % name
#rp = const.execute(sqlDel)
#rp.rowcount

In [8]:
old_name ='TIP'
new_name ='TIPH'

In [9]:
sqlUpd = '''
UPDATE dividend
SET name = "%s"
WHERE name = "%s"
'''
sqlUpd = sqlUpd % (new_name, old_name)
#rp = const.execute(sqlUpd)
#rp.rowcount

### End of Loop

In [10]:
sql = '''
SELECT D.name, Q4, Q3, Q2, Q1, DIVIDEND, shares, DIVIDEND * shares AS div_amt,
P.price AS current, shares*P.price AS amount,
FORMAT(DIVIDEND/P.price*100,2) AS yield, XDATE, PAIDDATE, ACTUAL
FROM dividend D
JOIN price P
WHERE D.name = P.name
AND P.date = "%s"'''
sql = sql % today
print(sql)


SELECT D.name, Q4, Q3, Q2, Q1, DIVIDEND, shares, DIVIDEND * shares AS div_amt,
P.price AS current, shares*P.price AS amount,
FORMAT(DIVIDEND/P.price*100,2) AS yield, XDATE, PAIDDATE, ACTUAL
FROM dividend D
JOIN price P
WHERE D.name = P.name
AND P.date = "2021-12-29"


In [11]:
div_price = pd.read_sql(sql, const)
div_price['yield'] = div_price['yield'].astype(float)
div_price['shares'] = div_price['shares'].astype(int)
div_price.shape

(79, 14)

In [12]:
div_price.nlargest(3, 'div_amt')[['name','div_amt']]

,name,div_amt
35,STA,105000.0
48,JASIF,95000.0
2,MCS,82200.0


In [13]:
y_ovr_X = div_price['yield'] >= 7.00
div_price[y_ovr_X].sort_values(['yield'],ascending=[False]).style.format(format_dict)

,name,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,yield,XDATE,PAIDDATE,ACTUAL
35,STA,1.75,1.25,1.25,1.00,5.25,"20,000","105,000.00",31.25,"625,000.00",16.80,2021-11-19,2021-12-08,0
33,STGT,2.00,1.25,1.25,0.00,4.50,0,0.00,30.00,0.00,15.00,2021-11-19,2021-12-07,0
46,NOBLE,0.50,0.00,0.35,0.00,0.85,"63,000","53,550.00",6.15,"387,450.00",13.82,2021-08-24,2021-09-07,0
24,TMT,0.50,0.00,0.60,0.00,1.10,"27,000","29,700.00",10.20,"275,400.00",10.78,2021-08-27,2021-09-10,0
2,MCS,0.60,0.37,0.40,0.00,1.37,"60,000","82,200.00",14.80,"888,000.00",9.26,2021-11-25,2021-12-09,0
48,JASIF,0.25,0.22,0.24,0.24,0.95,"100,000","95,000.00",10.30,"1,030,000.00",9.22,2021-11-19,2021-12-08,0
6,PDG,0.22,0.00,0.10,0.00,0.32,0,0.00,3.96,0.00,8.08,2021-03-15,2021-05-20,0
68,CPNCG,0.25,0.24,0.24,0.25,0.99,0,0.00,12.30,0.00,8.03,2021-11-15,2021-12-03,0
62,WIIK,0.09,0.00,0.10,0.00,0.19,0,0.00,2.38,0.00,7.98,2021-04-30,2021-05-21,0
61,TCAP,1.80,0.00,1.20,0.00,3.00,0,0.00,37.75,0.00,7.95,2021-09-13,2021-09-28,0


In [14]:
not_due = div_price['ACTUAL'] == 1
div_price[not_due].sort_values(['yield'],ascending=[False]).style.format(format_dict)

,name,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,yield,XDATE,PAIDDATE,ACTUAL


In [15]:
div_price[y_ovr_X & not_due].sort_values(['XDATE','name'],ascending=[True,True]).style.format(format_dict)

,name,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,yield,XDATE,PAIDDATE,ACTUAL


In [16]:
in_port = div_price['shares'] > 0
div_price[in_port].sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,yield,XDATE,PAIDDATE,ACTUAL
55,BCH,0.13,0.00,0.20,0.00,0.33,"15,000","4,950.00",20.30,"304,500.00",1.63,2021-08-31,2021-09-15,0
74,BGRIM,0.30,0.00,0.15,0.00,0.45,"9,000","4,050.00",40.00,"360,000.00",1.13,2021-03-11,2021-05-11,0
72,DCC,0.04,0.04,0.05,0.05,0.18,"40,000","7,360.00",2.84,"113,600.00",6.48,2021-11-19,2021-12-03,0
30,DIF,0.26,0.26,0.26,0.26,1.04,"60,000","62,640.00",14.00,"840,000.00",7.46,2021-11-11,2021-12-03,0
23,DOHOME,0.10,0.00,0.00,0.00,0.10,"8,000",800.00,23.20,"185,600.00",0.43,2021-05-21,2021-05-21,0
59,EPG,0.00,0.14,0.00,0.09,0.23,"14,000","3,220.00",11.10,"155,400.00",2.07,2021-11-25,2021-12-08,0
69,HREIT,0.17,0.17,0.17,0.17,0.69,"30,000","20,796.00",9.05,"271,500.00",7.66,2021-11-23,2021-12-23,0
10,IP,0.18,0.00,0.00,0.00,0.18,"7,500","1,387.50",19.40,"145,500.00",0.95,2022-04-08,2022-04-27,0
13,IVL,0.25,0.25,0.17,0.17,0.85,"18,000","15,300.00",43.00,"774,000.00",1.98,2021-11-25,2021-12-09,0
48,JASIF,0.25,0.22,0.24,0.24,0.95,"100,000","95,000.00",10.30,"1,030,000.00",9.22,2021-11-19,2021-12-08,0


In [17]:
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).style.format(format_dict)

,name,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,yield,XDATE,PAIDDATE,ACTUAL
35,STA,1.75,1.25,1.25,1.00,5.25,"20,000","105,000.00",31.25,"625,000.00",16.80,2021-11-19,2021-12-08,0
46,NOBLE,0.50,0.00,0.35,0.00,0.85,"63,000","53,550.00",6.15,"387,450.00",13.82,2021-08-24,2021-09-07,0
24,TMT,0.50,0.00,0.60,0.00,1.10,"27,000","29,700.00",10.20,"275,400.00",10.78,2021-08-27,2021-09-10,0
2,MCS,0.60,0.37,0.40,0.00,1.37,"60,000","82,200.00",14.80,"888,000.00",9.26,2021-11-25,2021-12-09,0
48,JASIF,0.25,0.22,0.24,0.24,0.95,"100,000","95,000.00",10.30,"1,030,000.00",9.22,2021-11-19,2021-12-08,0
69,HREIT,0.17,0.17,0.17,0.17,0.69,"30,000","20,796.00",9.05,"271,500.00",7.66,2021-11-23,2021-12-23,0
30,DIF,0.26,0.26,0.26,0.26,1.04,"60,000","62,640.00",14.00,"840,000.00",7.46,2021-11-11,2021-12-03,0
19,RATCH,1.25,0.00,2.00,0.00,3.25,"9,000","29,250.00",45.00,"405,000.00",7.22,2021-09-07,2021-09-23,0
78,RCL,0.50,1.50,1.50,0.00,3.50,"4,500","15,750.00",49.00,"220,500.00",7.14,2021-11-18,2021-12-03,0


In [18]:
dividend = div_price[y_ovr_X & in_port].div_amt.sum()
cost = div_price[y_ovr_X & in_port].amount.sum()
yld = round(dividend / cost * 100,2)
(dividend, cost, yld)

(493886.0, 4942850.0, 9.99)

In [19]:
file_name = 'DIV-OVR-X%.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
data_file, output_file, box_file

('../data/DIV-OVR-X%.csv',
 '\\Users\\User\\iCloudDrive\\DIV-OVR-X%.csv',
 '\\Users\\User\\Dropbox\\DIV-OVR-X%.csv')

In [20]:
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(data_file)
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(output_file)
div_price[y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).to_csv(box_file)

In [21]:
div_price[~y_ovr_X & in_port].sort_values(['yield','name'],ascending=[False,True]).style.format(format_dict)

,name,Q4,Q3,Q2,Q1,DIVIDEND,shares,div_amt,current,amount,yield,XDATE,PAIDDATE,ACTUAL
70,TISCO,6.30,0.00,0.00,0.00,6.30,"2,000","12,600.00",95.75,"191,500.00",6.58,2021-04-28,2021-05-17,0
72,DCC,0.04,0.04,0.05,0.05,0.18,"40,000","7,360.00",2.84,"113,600.00",6.48,2021-11-19,2021-12-03,0
76,MC,0.00,0.20,0.00,0.35,0.55,"10,000","5,500.00",9.05,"90,500.00",6.08,2021-11-04,2021-11-25,0
16,PTT,0.82,0.00,1.20,0.00,2.02,"9,000","18,180.00",38.25,"344,250.00",5.28,2021-09-29,2021-10-15,0
77,RJH,0.20,1.10,0.40,0.00,1.70,"4,500","7,650.00",33.00,"148,500.00",5.15,2021-11-22,2021-12-07,0
51,PTTGC,1.00,0.00,2.00,0.00,3.00,"5,400","16,200.00",58.50,"315,900.00",5.13,2021-09-03,2021-09-22,0
9,SCC,8.50,0.00,8.50,0.00,17.00,600,"10,200.00",388.00,"232,800.00",4.38,2021-08-13,2021-08-27,0
56,TU,0.40,0.00,0.45,0.00,0.85,"18,000","15,300.00",19.60,"352,800.00",4.34,2021-08-23,2021-09-07,0
32,SAT,0.48,0.00,0.40,0.00,0.88,"7,500","6,600.00",21.50,"161,250.00",4.09,2021-08-25,2021-09-07,0
14,NER,0.15,0.00,0.07,0.00,0.22,"36,000","7,920.00",7.05,"253,800.00",3.12,2021-08-23,2021-09-06,0


In [22]:
dividend = div_price[~y_ovr_X & in_port].div_amt.sum()
cost = div_price[~y_ovr_X & in_port].amount.sum()
yld = round(dividend / cost * 100,2)
(dividend, cost, yld)

(157867.5, 5215200.0, 3.03)